In [4]:
import tensorflow as tf
import sys
sys.path.append("../Data_Preprocessing/")
from Preprocess_Utilities import *

In [5]:
# restore trained model & training info into session
sess = tf.InteractiveSession()

saver = tf.train.import_meta_graph('./Result/models/res_serv_road_livi/res_serv_road_livi.meta')
saver.restore(sess, tf.train.latest_checkpoint('./Result/models/res_serv_road_livi/'))

OSError: File ./Result/models/res_serv_road_livi/res_serv_road_livi.meta does not exist.

In [ ]:
# read in dataset

In [ ]:
print(train_pred_neg.shape)
print()

for i in range(10):
    idx = np.random.randint(0, train_pred_neg.size)
    patch_idx = train_pred_neg[idx]
    while train_y[patch_idx] == 0:
        idx = np.random.randint(0, train_pred_neg.size)
        patch_idx = train_pred_neg[idx]        
    
    show_image_against_road(image=train_x[patch_idx].reshape(7, 28, 28), road=train_road_mask[patch_idx],
                            size=-1, figsize=(10,10))